In [68]:
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import (ConnectDrakeVisualizer, SceneGraph)
from pydrake.lcm import DrakeLcm
from pydrake.multibody.rigid_body_tree import (RigidBodyTree, AddModelInstancesFromSdfFile,
                                               FloatingBaseType,AddModelInstanceFromUrdfFile,AddFlatTerrainToWorld, RigidBodyFrame,AddModelInstanceFromUrdfStringSearchingInRosPackages)
from pydrake.multibody.multibody_tree import UniformGravityFieldElement, MultibodyTree,BodyIndex
from pydrake.multibody.multibody_tree.multibody_plant import MultibodyPlant
from pydrake.multibody.multibody_tree.parsing import AddModelFromSdfFile 
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator
from pydrake.common import (set_assertion_failure_to_throw_exception)
from pydrake.systems.controllers import (RbtInverseDynamicsController)
from pydrake.systems.primitives import (ConstantVectorSource, SignalLogger, TrajectorySource)
from pydrake.systems.meshcat_visualizer import MeshcatVisualizer
from pydrake.multibody.parsers import PackageMap
from pydrake.systems.framework import LeafSystem, PortDataType,BasicVector

from pydrake.multibody.rigid_body_plant import DrakeVisualizer,RigidBodyPlant
import numpy as np
import matplotlib.pyplot as plt
 
import os
import pydrake

from pydrake.systems.all import LcmSubscriberSystem, LcmPublisherSystem,AbstractValue
from pydrake.multibody.rigid_body_plant import ContactResults 
from pydrake.all import PySerializer
#from pydrake.all import  KinematicsResults

from systems.robot_coomand_to_rigidbody_converter import RobotCommandToRigidBodyPlantConverter
from systems.robot_state_encoder import RobotStateEncoder

In [69]:
from lcmt import *
model_path = os.path.join(os.getcwd(), 'Model/LittleDog.urdf')

dt = 0

lcm = DrakeLcm()
builder = DiagramBuilder()



In [70]:
# Build up your Robot World
rb_tree = RigidBodyTree()
world_frame = RigidBodyFrame("world_frame", rb_tree.world(), [0, 0, 0], [0, 0, 0])
AddFlatTerrainToWorld(rb_tree, 1000, 10)
robot_frame = RigidBodyFrame("robot_frame", rb_tree.world(), [0, 0, 0.5], [0, 0, 0])

# insert a robot from urdf files
pmap = PackageMap()
pmap.PopulateFromFolder(os.path.dirname(model_path))
AddModelInstanceFromUrdfStringSearchingInRosPackages(
        open(model_path, 'r').read(),
        pmap,
        os.path.dirname(model_path),
        FloatingBaseType.kRollPitchYaw,
        robot_frame,
        rb_tree)

{u'LittleDog': 0}

In [71]:
rb_tree.get_num_positions()

18

In [72]:
# RigidBodyPlant 
plant = builder.AddSystem(RigidBodyPlant(rb_tree, dt))
plant.set_name('plant')

# Robot command subscriber

#robot_command_subscriber = builder.AddSystem(LcmSubscriberSystem('ROBOT_COMMAND', PySerializer(littledog_command_t), lcm))
robot_command_subscriber = builder.AddSystem(LcmSubscriberSystem.Make('ROBOT_COMMAND', littledog_command_t, lcm))
robot_command_subscriber.set_name('robot_command_subscriber')

# Robot command to Plant Converter
robot_command_to_rigidbodyplant_converter = builder.AddSystem(RobotCommandToRigidBodyPlantConverter(rb_tree.actuators)) # input argu: rigidbody actuators
robot_command_to_rigidbodyplant_converter.set_name('robot_command_to_rigidbodyplant_converter')

# Visualizer
visualizer_publisher = builder.AddSystem(DrakeVisualizer(rb_tree, lcm ))
visualizer_publisher.set_name('visualizer_publisher')
visualizer_publisher.set_publish_period(1e-3);

# Robot State Encoder
robot_state_encoder = builder.AddSystem(RobotStateEncoder(rb_tree)) #force_sensor_info
robot_state_encoder.set_name('robot_state_encoder')

# Robot State Publisher
#robot_state_publisher = builder.AddSystem(LcmPublisherSystem('EST_ROBOT_STATE', PySerializer(robot_state_t), lcm))
robot_state_publisher = builder.AddSystem(LcmPublisherSystem.Make('EST_ROBOT_STATE', robot_state_t, lcm))
robot_state_publisher.set_name('robot_state_publisher')
robot_state_publisher.set_publish_period(1e-3);

 # Connect everything
builder.Connect(robot_command_subscriber.get_output_port(0),
                robot_command_to_rigidbodyplant_converter.robot_command_input_port())
builder.Connect(robot_command_to_rigidbodyplant_converter.desired_effort_output_port(),
                plant.get_input_port(0))
builder.Connect(plant.state_output_port(),
                visualizer_publisher.get_input_port(0))
# builder.Connect(plant.get_output_port(0),
#                 visualizer_publisher.get_input_port(0))

builder.Connect(plant.state_output_port(),
                robot_state_encoder.joint_state_results_input_port())
builder.Connect(robot_state_encoder.lcm_message_output_port(),
                robot_state_publisher.get_input_port(0))


position = np.array(msg.position)
        velocity = np.array(msg.velocity)
        effort = np.array(msg.effort)
        k_q_p = np.array(msg.k_q_p)
        k_q_i = np.array(msg.k_q_i)
        k_q_d = np.array(msg.k_q_d)
        k_qd_d = np.array(msg.k_qd_d)
        k_f_p = np.array(msg.k_f_p)
        ff_qd = np.array(msg.ff_qd)
        ff_qd_d = np.array(msg.ff_qd_d)
        ff_const = np.array(msg.ff_const)
        
        output_command = k_q_p *(q_d -q) + k_q_i * (q_d - q) +  
                         k_qd_p *(qd_d -qd) + k_f_p * (f_d - f) 
                         + ff_qd * qd + ff_f_d * fd +ff_const

In [73]:
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
simulator = Simulator(diagram, diagram_context)
simulator.set_publish_every_time_step(False)
simulation_time = 10


In [65]:

# simulator.set_target_realtime_rate(1)
# #simulator.Initialize()
# simulator.get_mutable_integrator().set_target_accuracy(1e-3)

# 

In [66]:
lcm.StartReceiveThread()
simulator.StepTo(simulation_time)
lcm.StopReceiveThread()